This code runs Deep Learning on nutau and nue samples created by Dasha.  It runs on multiple GPUs at once.

This code runs in a python 3.7 conda environment constructed as follows:
- conda create --name tf-gpu tensorflow-gpu keras scikit-learn matplotlib ipykernel nb_conda_kernels [Pillow]
- conda activate tf-gpu
(Pillow is for image manipulation for making heat maps, but I haven't got it to work yet.  Can remove from environment.)

To run with multiple GPUs on CyberLAMP you must specify the "nodes" and "gpus" qualifiers in the same chunk.  For example:
- qsub -I -A cyberlamp -l qos=cl_higpu -l nodes=1:ppn=1:gpus=4:shared -l mem=24gb -l walltime=4:00:00

In [1]:
from __future__ import division
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [2]:
import os

# Set which GPU to use.  This probably needs to be done before any other CUDA vars get defined.
# Use the command "nvidia-smi" to get association of a particular GPU with a particular number.
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]= "0,1,2"

In [3]:
c =0.299792458
n =1.3195
v=c/n 
print(v)

0.2272015596816976


In [4]:
info_d = []
info_s = []
data_d = []
data_s = []

for i in range(0,10):
    name_dd = "/home/dup193/work/double_pulse/data/Tau05to15PeV_00{0}_data.npy".format(i)
    name_di = "/home/dup193/work/double_pulse/data/Tau05to15PeV_00{0}_info.pkl".format(i)
    name_sd = "/home/dup193/work/double_pulse/data/Electron05to15PeV_00{0}_data.npy".format(i)
    name_si = "/home/dup193/work/double_pulse/data/Electron05to15PeV_00{0}_info.pkl".format(i)
    info_d_temp = pickle.load(open(name_di, "rb"))
    info_s_temp = pickle.load(open(name_si, "rb"))
    data_d_temp = np.load(name_dd ,allow_pickle=True,encoding='bytes')
    data_s_temp = np.load(name_sd ,allow_pickle=True,encoding='bytes')
    info_d = info_d + info_d_temp
    info_s = info_s + info_s_temp
    data_d.append(data_d_temp)    
    data_s.append(data_s_temp)
data_d = np.vstack(data_d)
data_s = np.vstack(data_s)
info_d = np.array(info_d)
info_s = np.array(info_s)

In [5]:
energy_l_d = []
energy_nu_d = []
charge_d = []
charge_st_d = []
for i in info_d:
    energy_l_d.append(i['tau_energy'])
    energy_nu_d.append(i['nu_energy'])
    charge_d.append(i['qtotal'])
    charge_st_d.append(i['strings']['charge'])

energy_l_d = np.array(energy_l_d)
energy_nu_d = np.array(energy_nu_d)
charge_d = np.array(charge_d)
charge_st_d = np.array(charge_st_d)

energy_l_s = []
energy_nu_s = []
charge_s = []
charge_st_s = []
for i in info_s:
    energy_l_s.append(i['tau_energy'])
    energy_nu_s.append(i['nu_energy'])
    charge_s.append(i['qtotal'])
    charge_st_s.append(i['strings']['charge'])

energy_l_s = np.array(energy_l_s)
energy_nu_s = np.array(energy_nu_s)
charge_s = np.array(charge_s)
charge_st_s = np.array(charge_st_s)
 
print(len(energy_l_d),len(energy_nu_d),len(charge_d),len(charge_st_d))
print(len(energy_l_s),len(energy_nu_s),len(charge_s),len(charge_st_s))

10062 10062 10062 10062
24019 24019 24019 24019


In [6]:
def Plot(arr1,arr2, bs =100, r = [0,100], lab = "Charge"):
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111)
    weights1 = np.ones_like(arr1)/float(len(arr1))
    weights2 = np.ones_like(arr2)/float(len(arr2))
    #ax.set_yscale(scale)
    #ax.set_title(name, fontsize = 14)
    ax.text(0.5,0.95,'#Events single '+str(len(arr1)), transform=ax.transAxes, color = "black",fontsize=8)
    ax.text(0.5,0.90,'#Events double '+str(len(arr2)), transform=ax.transAxes, color = "black",fontsize=8)
    ax.set_xlabel(lab, fontsize = 14)                                                              
    ax.set_ylabel("FractionEvents", fontsize = 14)                                                   
    ax.hist(arr1, bins =bs, weights = weights1, range = r, histtype = 'step',edgecolor ='r', fill= False, label = 'Single')
    ax.hist(arr2, bins =bs, weights = weights2, range = r, histtype = 'step',edgecolor ='b', fill= False, label = 'Double')


In [7]:
#Plot(energy_l_s/1000000,energy_l_d/1000000, bs =50, r = [0.4,1.6], lab = "energy of lepton, GeV")

In [8]:
#Plot(energy_nu_s/1000000,energy_nu_d/1000000, bs =50, r = [0.4,3.6], lab = "energy of neutrino, GeV")

In [9]:
#Plot(charge_s,charge_d,bs = 50, r= [0,200000], lab = "total charge, Pe")

In [10]:
#Plot(charge_st_s,charge_st_d,bs = 50, r= [0,200000], lab = "string charge, Pe")

In [11]:
print(info_d.shape, data_d.shape)
print(info_s.shape, data_s.shape)
print(type(data_d_temp[0][0][0]))

(10062,) (10062, 300, 60)
(24019,) (24019, 300, 60)
<class 'numpy.float64'>


In [12]:
print(np.amax(data_d))
print(np.amin(data_d))
print(np.amax(data_s))
print(np.amin(data_s))

6.405907616717559e-09
-5.962297593924894e-11
6.883592770365534e-09
-6.128851422642174e-11


In [13]:
from keras.utils import to_categorical
label_d = [[0,1]]*len(data_d)
label_s = [[1,0]]*len(data_s)
label_d = np.array(label_d)
label_s = np.array(label_s)
print(label_d.shape)
print(label_s.shape)
print(label_d[0])
print(label_s[0])

Using TensorFlow backend.


(10062, 2)
(24019, 2)
[0 1]
[1 0]


In [14]:
from sklearn.utils import shuffle

In [15]:
data = np.concatenate((data_d, data_s), axis = 0) 
label = np.concatenate((label_d, label_s), axis = 0) 
data, label = shuffle(data, label, random_state =12)

train_data = data[:24000]
train_label = label[:24000]
train_data = train_data.reshape((len(train_data),300,60,1))
train_data = train_data.astype('float32')/10**-8
mean = np.mean(train_data)
std = np.std(train_data)
print(mean,std)
train_data = train_data - mean
train_data = train_data/std

valid_data = data[24000:28000]
valid_label = label[24000:28000]
valid_data = valid_data.reshape((len(valid_data),300,60,1))
valid_data = valid_data.astype('float32')/10**-8
valid_data = valid_data - mean
valid_data = valid_data/std

test_data = data[28000:]
test_label = label[28000:]
test_data = test_data.reshape((len(test_data),300,60,1))
test_data = test_data.astype('float32')/10**-8
test_data = test_data - mean
test_data = test_data/std


0.0026542188 0.022200854


In [16]:
print(np.amax(train_data))
print(np.amin(train_data))

30.886427
-0.3950856


In [17]:
from keras import layers
from keras import models

In [18]:
import tensorflow as tf
from keras.utils import multi_gpu_model
from keras import optimizers

with tf.device('/cpu:0'):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(300, 60, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(2, activation='softmax'))
    
parallel_model = multi_gpu_model(model,gpus=3)
sgd = optimizers.SGD(lr=0.01)
parallel_model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
history = parallel_model.fit(train_data,train_label, epochs=300, validation_data=(valid_data,valid_label), batch_size =192)

W0815 16:56:30.684806 139942403921664 deprecation_wrapper.py:119] From /home/dfc13/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0815 16:56:30.687193 139942403921664 deprecation_wrapper.py:119] From /home/dfc13/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0815 16:56:30.690948 139942403921664 deprecation_wrapper.py:119] From /home/dfc13/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0815 16:56:30.710994 139942403921664 deprecation_wrapper.py:119] From /home/dfc13/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. P

Train on 24000 samples, validate on 4000 samples
Epoch 1/30
24000/24000 [==============================] - 13s 550us/step - loss: 0.6238 - acc: 0.7015 - val_loss: 0.6118 - val_acc: 0.7052
Epoch 2/30
24000/24000 [==============================] - 5s 201us/step - loss: 0.6030 - acc: 0.7075 - val_loss: 0.6067 - val_acc: 0.7052
Epoch 3/30
24000/24000 [==============================] - 5s 215us/step - loss: 0.6003 - acc: 0.7076 - val_loss: 0.6055 - val_acc: 0.7052
Epoch 4/30
24000/24000 [==============================] - 6s 231us/step - loss: 0.6000 - acc: 0.7077 - val_loss: 0.6041 - val_acc: 0.7052
Epoch 5/30
24000/24000 [==============================] - 5s 222us/step - loss: 0.5982 - acc: 0.7078 - val_loss: 0.6037 - val_acc: 0.7052
Epoch 6/30
24000/24000 [==============================] - 5s 190us/step - loss: 0.5974 - acc: 0.7075 - val_loss: 0.6035 - val_acc: 0.7047
Epoch 7/30
24000/24000 [==============================] - 5s 228us/step - loss: 0.5962 - acc: 0.7078 - val_loss: 0.6040 - 

In [19]:
import json
history_dict = history.history # Get the dictionary containing each metric and the loss for each epoch
json.dump(history_dict, open('/data/dfc13/DoublePulse/Models/DP_model_10f-3.json', 'w')) # Save it under the form of a json file
#
model.save('/data/dfc13/DoublePulse/Models/DP_model_10f-3.h5') # Save the model
#
ResultsFile = open('/data/dfc13/DoublePulse/Models/DP_model_10f-3.txt','w') # File for saving the results of the fit.

# 8: multiple conv2d layers a la textbook (32 (3,3) then 64, 128, 128, dropout 0.3, dense 256(relu), 64(relu), 2(softmax)
# 8b: same as 8 but with 50 trials
# 8b[not c didn't update the line above]: same as 8b but with dropout 0.5.
# 8c: changed SGC "lr" from 0.01->0.001.  This smoothed things out a lot.  Sort of.  But had 0% accuracy for nutaus.
# 8d: changed SGC "lr" from 0.001->0.005.
# 8e: decreased size of training set to 24k, increased size of validation set to 4k (24k:28k) and increased test set (28k:)
# 8f: changed SGC "lr" from 0.005->0.01.  200 epochs. 97%/37%.
# 8g: lr=0.02, 50 epochs: 98%/29%.

# 9a: changed dense 256 -> 512.  50 epochs. 100%/22%
# 9b: same as above, 200 epochs. 97%/39%.
# 9c: same as above, lr = 0.01.  200 epochs. 97%/36%.
# 9d: same as above, 500 epochs batch_size = 64. 96%/40%.
# 10a: Testing selection of particular GPU.  20 epochs, batch_size = 128.
# 10b-3: Testing parallelization.  50 epochs, batch_size = 128, 3 GPUs (about 6s/epoch): 99%/19%.
# 10c-3: Testing parallelization.  50 epochs, batch_size = 64*3=192, 3 GPUs (about 4.5s/epoch): 100%/11%.
# 10d-3: Testing parallelization.  100 epochs, batch_size = 64*3=192, 3 GPUs (about 4s/epoch): 98%/29%.
# 10e-3: Testing parallelization.  500 epochs, batch_size = 64*3=192, 3 GPUs (about 4s/epoch): 98%/29%; 98%/56%.
# 10f-3: Parallel.  300 epochs, batch_size = 64*3=192, 3 GPUs. X%/Y%.

In [20]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

#plt.plot(epochs, acc, 'bo', label='Training acc')
#plt.plot(epochs, val_acc, 'b', label='Validation acc')
#plt.title('Training and validation accuracy')
#plt.legend()

#plt.figure()

#plt.plot(epochs, loss, 'bo', label='Training loss')
#plt.plot(epochs, val_loss, 'b', label='Validation loss')
#plt.title('Training and validation loss')
#plt.legend()

#plt.show()

In [21]:
result = parallel_model.evaluate(test_data,test_label)
print(result)

6081/6081 [==============================] - 2s 263us/step
[0.6084520763922358, 0.6938003617826015]


In [22]:
from sklearn.metrics import classification_report, confusion_matrix

In [23]:
test_pred = parallel_model.predict(test_data)
matrix = confusion_matrix(test_label.argmax(axis=1), test_pred.argmax(axis=1))
report = classification_report(test_label.argmax(axis=1), test_pred.argmax(axis=1))

In [24]:
#print("True Positive",tp)
#print("True Negative",tn)
#print("False Positive",fp)
#print("False Negative",fn)

ResultsFile.write(repr(matrix))
ResultsFile.write('\n')
ResultsFile.write(report)
ResultsFile.close()

print(matrix)
print(report)

[[4215    1]
 [1861    4]]
              precision    recall  f1-score   support

           0       0.69      1.00      0.82      4216
           1       0.80      0.00      0.00      1865

   micro avg       0.69      0.69      0.69      6081
   macro avg       0.75      0.50      0.41      6081
weighted avg       0.73      0.69      0.57      6081

